## Importing Libraries

In [1]:
import pandas as pd
import numpy as np
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from tensorflow.python.keras.models import sequential
from keras.layers import Dense, Embedding, LSTM, GRU
from keras.layers.embeddings import Embedding

Using TensorFlow backend.


In [2]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [3]:
cd /content/drive/My\ Drive/Colab\ Notebooks/Sentiment\ Analysis

/content/drive/My Drive/Colab Notebooks/Sentiment Analysis


In [0]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)
pd.set_option('display.max_rows', None)

## Importing Data

In [5]:
df = pd.DataFrame()
df = pd.read_csv('SentimentProcessedText1.csv', encoding='utf-8')
df.head(10)

,tweet,sentiment,Subjectivity
0,get first swigy orders use code grab checkout max discount order tca,2,0.333333
1,never use food service deliver worst quality food help desk executive politnes,0,1.000000
2,hi want refund zomato gold membership bought separate set tampc,1,0.000000
3,horrible experience swigy today received bad quality food subway word custome,0,0.833333
4,policy change even allowed pl buy zomato gold membership,1,0.000000
5,amazing food delivery ap never seen bz deliver food even hrs order,2,0.900000
6,review paji amp sons paschim vihar,1,0.000000
7,hunger tweets help handle get swigy ap,1,0.000000
8,situations usually open swigy ap order something regret next morning,1,0.250000
9,scrolling fodpanda could decide eat closed ap dia keluar notification ni ay,0,0.100000


In [6]:
print("Shape of Dataframe: ", df.shape)
print(df.columns)

Shape of Dataframe:  (5923, 3)
Index(['tweet', 'sentiment', 'Subjectivity'], dtype='object')


**Splitting into train and test set**

In [0]:
X_train = df.loc[:4739, 'tweet'].values
y_train = df.loc[:4739, 'sentiment'].values
X_test = df.loc[4740:, 'tweet'].values
y_test = df.loc[4740:, 'sentiment'].values

* Taking 20 percent of data as Validation set

In [11]:
X = np.concatenate((X_train, X_test), axis=0)
y = np.concatenate((y_train, y_test), axis=0)

# summarize size
print("Training data: ")
print("Dimensions of X: ", X.shape)
print("Dimensions of y: ",y.shape)

Training data: 
Dimensions of X:  (5923,)
Dimensions of y:  (5923,)


In [12]:
# y_train_ = pd.DataFrame({'Column1': data[:, 0], 'Column2': data[:, 1]})
y_train_dummies = pd.get_dummies(y_train)
print(y_train_dummies)
y_test_dummies = pd.get_dummies(y_test)
print(y_test_dummies)

      0  1  2
0     0  0  1
1     1  0  0
2     0  1  0
3     1  0  0
4     0  1  0
5     0  0  1
6     0  1  0
7     0  1  0
8     0  1  0
9     1  0  0
10    1  0  0
11    0  0  1
12    0  0  1
13    0  1  0
14    1  0  0
15    0  0  1
16    0  0  1
17    0  1  0
18    0  0  1
19    0  1  0
20    1  0  0
21    0  0  1
22    0  1  0
23    0  1  0
24    0  1  0
25    1  0  0
26    1  0  0
27    0  1  0
28    1  0  0
29    0  0  1
30    0  0  1
31    1  0  0
32    0  1  0
33    0  0  1
34    1  0  0
35    0  1  0
36    0  0  1
37    0  0  1
38    0  1  0
39    0  1  0
40    1  0  0
41    0  0  1
42    0  1  0
43    0  1  0
44    1  0  0
45    0  1  0
46    1  0  0
47    0  0  1
48    1  0  0
49    0  0  1
50    0  0  1
51    0  0  1
52    1  0  0
53    0  1  0
54    0  0  1
55    0  1  0
56    1  0  0
57    0  0  1
58    1  0  0
59    0  1  0
60    0  1  0
61    0  1  0
62    0  1  0
63    0  1  0
64    0  1  0
65    0  1  0
66    0  1  0
67    0  1  0
68    0  0  1
69    0  1  0
70    

* Since there are three categories 2, 1, 0 in our target variable that is sentiment variable we will have to feed matrix of one hot encoded vectors into a LSTM

In [13]:
X_test

array(['well swigy made starve literally customer care says food delivered within min deli',
       'deal arrived going eat alone thank fodpanda superseptemberdeals',
       'yup good know restaurant lacks proper hygiene conditions state yo',
       ..., 'guy wrote whole review kfc zomato unready',
       'register swigy nhlpv earn rs download',
       'exactly wait zomato person likeorderd looking towards door'],
      dtype=object)

In [14]:
tokenizer_obj = Tokenizer()
total_reviews = X
print("total number of tweets: ", len(total_reviews))

tokenizer_obj.fit_on_texts(total_reviews)
print(tokenizer_obj.word_index)

# pad sequences
max_length = max([len(s.split()) for s in total_reviews]) # try other options like mean
print("max length:" , max_length)

total number of tweets:  5923
{'zomato': 1, 'order': 2, 'swigy': 3, 'fodpanda': 4, 'food': 5, 'delivery': 6, 'gold': 7, 'service': 8, 'get': 9, 'ordered': 10, 'hi': 11, 'please': 12, 'customer': 13, 'hey': 14, 'us': 15, 'time': 16, 'ap': 17, 'one': 18, 'guys': 19, 'use': 20, 'share': 21, 'restaurant': 22, 'rs': 23, 'delivered': 24, 'got': 25, 'people': 26, 'number': 27, 'like': 28, 'kindly': 29, 'money': 30, 'change': 31, 'still': 32, 'review': 33, 'supo': 34, 'details': 35, 'code': 36, 'call': 37, 'never': 38, 'refund': 39, 'worst': 40, 'amp': 41, 'team': 42, 'offer': 43, 'customers': 44, 'new': 45, 'today': 46, 'look': 47, 'orders': 48, 'really': 49, 'chat': 50, 'experience': 51, 'checked': 52, 'even': 53, 'want': 54, 'also': 55, 'policy': 56, 'first': 57, 'pathetic': 58, 'deliver': 59, 'good': 60, 'online': 61, 'care': 62, 'hours': 63, 'membership': 64, 'bad': 65, 'would': 66, 'via': 67, 'hour': 68, 'getting': 69, 'back': 70, 'know': 71, 'id': 72, 'waiting': 73, 'check': 74, 'using'

* The tokenizer function forms a dictionary with key as words present in our text and value as some integer

In [15]:
# define vocabulary size
vocab_size = len(tokenizer_obj.word_index) + 1
print("vocab size:", vocab_size)

X_train_tokens =  tokenizer_obj.texts_to_sequences(X_train)
X_test_tokens = tokenizer_obj.texts_to_sequences(X_test)

print(X_train_tokens)
X_train_pad = pad_sequences(X_train_tokens, maxlen=max_length, padding='post')
X_test_pad = pad_sequences(X_test_tokens, maxlen=max_length, padding='post')

vocab size: 8187
[[9, 57, 3, 48, 20, 36, 535, 604, 758, 194, 2, 2508], [38, 20, 5, 8, 59, 40, 241, 5, 76, 2509, 160, 3734], [11, 54, 39, 1, 7, 64, 342, 1552, 536, 302], [314, 51, 3, 46, 99, 65, 241, 5, 499, 1150, 1553], [56, 31, 53, 605, 359, 258, 1, 7, 64], [360, 5, 6, 17, 38, 500, 3735, 59, 5, 53, 242, 2], [33, 3736, 41, 3737, 3738, 3739], [900, 1926, 76, 901, 9, 3, 17], [3740, 1312, 424, 3, 17, 2, 189, 696, 282, 606], [1554, 4, 91, 1015, 259, 537, 17, 1313, 3741, 759, 1016, 3742], [11, 607, 158, 251, 66, 28, 1017], [19, 49, 60, 1018, 1019, 2510, 212], [19, 49, 60, 1018, 1019, 2510, 212, 1020], [71, 1021, 3, 1018, 3743, 96], [180, 1151, 361, 538, 6, 343, 902, 3744, 2511], [32, 93, 283, 2, 72, 260, 303, 1022, 13, 2512], [9, 57, 3, 2, 20, 36, 2513, 226, 2, 647, 758, 194, 2, 2508], [74, 2514, 1927, 1928, 2515, 74, 1314, 1555, 112], [31, 1315, 7, 539, 195, 81, 1556], [1023, 903, 304, 3745, 3746, 2516, 1557, 3747, 83, 1929], [60, 1316, 40, 13, 34, 95, 48, 24, 151, 1317, 3748, 25], [1558, 

* X_train_tokens represent sequences of integer created from each tweet through integer mapping of words
* We are also padding tweets to match the length of max length of a tweet

In [16]:
EMBEDDING_DIM = 100

print('Build model...')

model = Sequential()
model.add(Embedding(vocab_size, EMBEDDING_DIM, input_length=max_length))
model.add(LSTM(32,  dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))

# try using different optimizers and different optimizer configs
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print('Summary of the built model...')
print(model.summary())


# Build the model
# print('Build model...')
# model = Sequential()
# model.add(LSTM(hidden_nodes, return_sequences=False, input_shape=(word_vec_length, char_vec_length)))
# model.add(Dropout(0.2))
# model.add(Dense(units=output_labels))
# model.add(Activation('softmax'))
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

# batch_size=1000
# model.fit(train_x, train_y, batch_size=batch_size, epochs=10, validation_data=(validate_x, validate_y))


Build model...
Summary of the built model...
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 23, 100)           818700    
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                17024     
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 99        
Total params: 835,823
Trainable params: 835,823
Non-trainable params: 0
_________________________________________________________________
None


* We are training embedding along with our model rather than using a pre trained word2vec model
* Since our dataset is small we are using just one hidden layer and one dense layer

In [17]:
print('Train...')

model.fit(X_train_pad, y_train_dummies, batch_size=128, epochs=25, validation_data=(X_test_pad, y_test_dummies), verbose=1)

Train...


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 4740 samples, validate on 1183 samples
Epoch 1/25
4740/4740 [==============================] - 5s 1ms/step - loss: 1.0636 - accuracy: 0.4504 - val_loss: 1.0557 - val_accuracy: 0.4539
Epoch 2/25
4740/4740 [==============================] - 3s 635us/step - loss: 0.9839 - accuracy: 0.4916 - val_loss: 0.8866 - val_accuracy: 0.5520
Epoch 3/25
4740/4740 [==============================] - 3s 658us/step - loss: 0.6579 - accuracy: 0.6871 - val_loss: 0.7288 - val_accuracy: 0.6669
Epoch 4/25
4740/4740 [==============================] - 3s 658us/step - loss: 0.5125 - accuracy: 0.7348 - val_loss: 0.8162 - val_accuracy: 0.6374
Epoch 5/25
4740/4740 [==============================] - 3s 637us/step - loss: 0.4458 - accuracy: 0.7555 - val_loss: 0.7890 - val_accuracy: 0.6644
Epoch 6/25
4740/4740 [==============================] - 3s 661us/step - loss: 0.4166 - accuracy: 0.7646 - val_loss: 0.7609 - val_accuracy: 0.6729
Epoch 7/25
4740/4740 [==============================] - 3s 638us/step - loss: 

In [18]:
print('Testing...')
score, acc = model.evaluate(X_test_pad, y_test_dummies, batch_size=128)
print('Test score:', score)
print('Test accuracy:', acc)
print("Accuracy: {0:.2%}".format(acc))


Testing...
1183/1183 [==============================] - 0s 79us/step
Test score: 0.790447805155563
Test accuracy: 0.8529163002967834
Accuracy: 85.29%


In [23]:
from sklearn.metrics import confusion_matrix
yhat_classes = model.predict_classes(X_test_pad, verbose=0)
print(yhat_classes)
matrix = confusion_matrix(y_test, yhat_classes)
matrix

[1 1 2 ... 2 1 2]


array([[219,  23,  35],
       [ 18, 485,  34],
       [ 25,  39, 305]])

In [0]:
# from yellowbrick.classifier import ClassificationReport

# # Specify the target classes
# classes = ['0', '1', '2']
# visualizer = ClassificationReport(model, classes=classes, support=True)

# visualizer.fit(X_train_pad, y_train_dummies)        # Fit the visualizer and the model
# visualizer.score(X_test_pad, y_test_dummies)        # Evaluate the model on the test data
# visualizer.show()       

In [33]:
from sklearn.metrics import classification_report
report = classification_report(y_test, yhat_classes, labels=[0,1,2], target_names=['Negative Sentiment', 'Neutral Sentiment', 'Positive Sentiment'], sample_weight=None, digits=2, output_dict=True, zero_division='warn')
df_report = pd.DataFrame(report).transpose()
df_report

,precision,recall,f1-score,support
Negative Sentiment,0.835878,0.790614,0.812616,277.000000
Neutral Sentiment,0.886654,0.903166,0.894834,537.000000
Positive Sentiment,0.815508,0.826558,0.820996,369.000000
accuracy,0.852916,0.852916,0.852916,0.852916
macro avg,0.846013,0.840113,0.842815,1183.000000
weighted avg,0.852573,0.852916,0.852551,1183.000000


In [0]:
#Let us test some  samples
test_sample_1 = "food delivey satisfactory"
test_sample_2 = "Good food"
test_sample_3 = "Maybe like fast delivery."
test_sample_4 = "unpleasant taste skip  watch another delivery platform"
test_sample_5 = "like action food  good"
# test_sample_6 = "Bad movie!"
# test_sample_7 = "Not a good movie!"
# test_sample_8 = "This movie really sucks! Can I get my money back please?"
test_samples = [test_sample_1, test_sample_2, test_sample_3, test_sample_4, test_sample_5]

test_samples_tokens = tokenizer_obj.texts_to_sequences(test_samples)
test_samples_tokens_pad = pad_sequences(test_samples_tokens, maxlen=max_length)

#predict
ans = model.predict(x=test_samples_tokens_pad)
ans

array([[5.7174283e-04, 9.9789596e-01, 1.5322760e-03],
       [7.7297492e-04, 9.9708623e-01, 2.1408489e-03],
       [1.1327030e-03, 9.9632865e-01, 2.5385658e-03],
       [9.4455772e-04, 9.9664390e-01, 2.4116479e-03],
       [1.8711305e-03, 9.9489647e-01, 3.2323068e-03]], dtype=float32)

In [0]:
# x = np.amax(ans, 1)
# x

In [0]:


# # In[ ]:


# #let us check how the model predicts
# classes = model.predict(X_test_pad[:10], batch_size=128)
# print(classes)

In [0]:

# for i in range (0,10):
#     if(classes[i] > 0.5 and y_test[i] == 1 or (classes[i] <= 0.5 and y_test[i] == 0)):
#         print( classes[i], y_test[i], " Right prdiction")
#     else :
#         print( classes[i], y_test[i], " Wrong prdiction")

